# ...in C

## PubMed API in C

by Cyrus Gomes and Dr. Vincent Scalfani

These recipe examples were tested on July 25, 2023.

**NCBI Entrez Programming Utilities documentation:** https://www.ncbi.nlm.nih.gov/books/NBK25501/

**Please see NCBI's Data Usage Policies and Disclaimers:** https://www.ncbi.nlm.nih.gov/home/about/policies/

### Setup

First, install the CURL and jq package by typing the following command in the terminal:

In [ ]:
!sudo apt install curl jq libcurl4-openssl-dev

First we set a directory where we want the PubMed directory for our projects to be created 

In [1]:
!mkdir PubMed

We change the directory to the folder we created

In [ ]:
%cd PubMed


### 1. Basic PubMed API call

We initialize a folder for the current project that we are working on. And then change to that directory

In [3]:
!mkdir basic_api_call

In [ ]:
%cd basic_api_call

Then we utilize the following command (%%file) to create the following makefile which will compile our program and create an executable.

In [ ]:
%%file makefile

#sets the variable CC to gcc, which is used to build the program
CC=gcc

#enable debugging information and enable all compiler warnings
CFLAGS=-g -Wall

#sets the bin variable as the name of the binary file we are creating
BIN=api_call

#creates the binary file with the name we put
all: $(BIN)

#maps any file ending in .c to a binary executable. 
#"$<" represents the .c file and "$@" represents the target binary executable
%: %.c

#compiles the .c file using the gcc compiler with the CFLAGS and links 
#resulting binary with the CURL library
	$(CC) $(CFLAGS) $< -o $@ -lcurl

#clean target which removes specific files
clean:

#removes the binary file and an ".dSYM" (debug symbols for debugging) directories
#the RM command used -r to remove directories and -f to force delete
	$(RM) -rf $(BIN) *.dSYM


The command is used again to create our .c file which contains the code for the program

In [ ]:
%%file api_call.c

#include <curl/curl.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

/*CURL program that retrieves JSON data from the Pub Chem API
This program allows custom indicator data set to be used*/


/* We are going to be inputting the custom ID like this: ./api_call -i 42342346
If the arguments are missing then we use the default: "27933103"
*/

int main (int argc, char* argv[]){
    
    //if arguments are invalid just return
    if (argc > 5){                                                                                      
        printf("Error. Please try again correctly.\n");
        return -1;
    }

    //default indicator code
    char indicator[100] = {}; 

    //if there is ./api_call or -i
    if ((argc == 1) || ((argc == 2) && ((strcmp(argv[1], "-i")==0)))){
        //these arguments run the default parameters and keeps the codes as they are
        strcat(indicator, "27933103");
    }

    //if there is ./api_call -i 34813985
    else if ((argc == 3) && (strcmp(argv[1], "-i")==0)){
        //only the country code is changed
        strcat(indicator, argv[2]);
    }

    else{
        printf("usage: ./api_call [-i] indicator\n\n");
        printf("the custom_ID program is used to retrieve json data from the Pub Med API\n\n");
        printf("optional arguments\n");
        printf("\t -i ID    optional custom PubMed ID; default is '27933103'\n");
        return -1;
    }

    //initializes the CURL HTTP connection
    CURL *curl = curl_easy_init();

    //bits of the url that are joined together later
    char api[] = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&";                                                                     
    char type1[] = "id=";                          
    char url[1000];
    char label[] = "&retmode=json";

    //checks if CURL initialization is a success or not
    if (!curl){                                                                                         
        fprintf(stderr, "init failed\n");
        return EXIT_FAILURE;
    }
        
    //combines all the bits to produce a functioning url
    sprintf(url, "%s%s%s%s", api, type1 , indicator, label);                                             
                                          

    //sets the url to which the HTTP request will be sent to
    //first parameter is for the initialized curl HTTP request, second for the option to be set, and third for the value to be set
    curl_easy_setopt(curl, CURLOPT_URL, url);

    //if result is not retrieved then output error
    CURLcode result = curl_easy_perform(curl);

    //if result is not retrieved then output error
    if (result != CURLE_OK){                                                                            
        fprintf(stderr, "download problem: %s\n", curl_easy_strerror(result));
    }

    //deallocates memory for the CURL connection
    curl_easy_cleanup(curl);                                                                            
    return EXIT_SUCCESS;
}

The folowing program is run, and an executable is created after using the following command:

In [ ]:
!make

The article we are requesting has PubMed ID: 27933103

To print the following json data we do the following:

In [8]:
!./api_call | jq '.'

{
  "header": {
    "type": "esummary",
    "version": "0.3"
  },
  "result": {
    "uids": [
      "27933103"
    ],
    "27933103": {
      "uid": "27933103",
      "pubdate": "2016",
      "epubdate": "2016 Nov 23",
      "source": "J Cheminform",
      "authors": [
        {
          "name": "Scalfani VF",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Williams AJ",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Tkachenko V",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Karapetyan K",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Pshenichnov A",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Hanson RM",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Liddie JM",
          "autht

To output the data for multiple ids from the PubMed API, we enter the following command:

In [9]:
!./api_call -i "34813985,34813140" | jq '.'

{
  "header": {
    "type": "esummary",
    "version": "0.3"
  },
  "result": {
    "uids": [
      "34813985",
      "34813140"
    ],
    "34813985": {
      "uid": "34813985",
      "pubdate": "2022 Jan",
      "epubdate": "2021 Nov 8",
      "source": "Cell Calcium",
      "authors": [
        {
          "name": "Fernández-Morales JC",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Xia Y",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Renzo TJ",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Zhang XH",
          "authtype": "Author",
          "clusterid": ""
        },
        {
          "name": "Morad M",
          "authtype": "Author",
          "clusterid": ""
        }
      ],
      "lastauthor": "Morad M",
      "title": "Mutation in RyR2-FKBP Binding site alters Ca(2+) signaling modestly but increases \"arrhythmogenesis\

To output the data for multiple ids from the PubMed API, we enter the following command:

In [10]:
!./api_call | jq '.["result"]["27933103"]["authors"][]'

{
  "name": "Scalfani VF",
  "authtype": "Author",
  "clusterid": ""
}
{
  "name": "Williams AJ",
  "authtype": "Author",
  "clusterid": ""
}
{
  "name": "Tkachenko V",
  "authtype": "Author",
  "clusterid": ""
}
{
  "name": "Karapetyan K",
  "authtype": "Author",
  "clusterid": ""
}
{
  "name": "Pshenichnov A",
  "authtype": "Author",
  "clusterid": ""
}
{
  "name": "Hanson RM",
  "authtype": "Author",
  "clusterid": ""
}
{
  "name": "Liddie JM",
  "authtype": "Author",
  "clusterid": ""
}
{
  "name": "Bara JE",
  "authtype": "Author",
  "clusterid": ""
}


To output only the author names:

In [11]:
!./api_call | jq '.["result"]["27933103"]["authors"][]["name"]'

"Scalfani VF"
"Williams AJ"
"Tkachenko V"
"Karapetyan K"
"Pshenichnov A"
"Hanson RM"
"Liddie JM"
"Bara JE"


To output the source name from the PubMed API, we enter the following command:

In [12]:
!./api_call -i 34813072 | jq '.["result"]["34813072"]["source"]'

"Methods Mol Biol"


Here, we output the source name for multiple ids:

In [13]:
!idList=('34813985' '34813932' '34813684' '34813661' '34813372' '34813140' '34813072');\
for id in "${idList[@]}"; \
do ./api_call -i "$id" | jq --arg location "$id" '.["result"][$location]["source"]'; \
sleep 1 ; \
done

"Cell Calcium"
"Methods"
"FEBS J"
"Dev Growth Differ"
"CRISPR J"
"Chembiochem"
"Methods Mol Biol"


### 3. PubMed API Calls with Requests & Parameters


We go back to our original directory

In [ ]:
%cd ..

We initialize a folder for the current project that we are working on. And then change to that directory

In [15]:
!mkdir api_request_parameter

We then change directory to the project that we are working on

In [ ]:
%cd api_request_parameter

In [ ]:
%%file makefile


#sets the variable CC to gcc, which is used to build the program
CC=gcc

#enable debugging information and enable all compiler warnings
CFLAGS=-g -Wall

#sets the bin variable as the name of the binary file we are creating
BIN=api_req_par

#creates the binary file with the name we put
all: $(BIN)

#maps any file ending in .c to a binary executable. 
#"$<" represents the .c file and "$@" represents the target binary executable
%: %.c

#compiles the .c file using the gcc compiler with the CFLAGS and links 
#resulting binary with the CURL library
	$(CC) $(CFLAGS) $< -o $@ -lcurl

#clean target which removes specific files
clean:

#removes the binary file and an ".dSYM" (debug symbols for debugging) directories
#the RM command used -r to remove directories and -f to force delete
	$(RM) -rf $(BIN) *.dSYM


The command is used again to create our .c file which contains the code for the program

In [ ]:
%%file api_req_par.c

#include <curl/curl.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

/*CURL program that retrieves JSON data from the PubMed API
This program allows custom request to be used along with the parameter*/


/* We are going to be inputting the custom database and query like this: ./api_req_par -d "pubmed" -q "neuroscience+intervention+learning"
If the arguments are missing then we use the default: "pubmed" "neuroscience"
*/

int main (int argc, char* argv[]){
    
    //if arguments are invalid just return
    if (argc > 5){                                                                                      
        printf("Error. Please try again correctly.\n");
        return -1;
    }

    //default parameter and request codes
    char parameter[100] = {};
    char request[500] = {}; 

    //if there is ./api_req_par -d/-q
    if ((argc == 1) || ((argc == 2) && ((strcmp(argv[1], "-d")==0) || (strcmp(argv[1], "-q")==0)))){
        //these arguments run the default parameters and keeps the codes as they are
        strcat(parameter,"pubmed");
        strcat(request, "neuroscience");
    }

    //if there is ./api_req_par -d "pubmed"
    else if ((argc == 3) && (strcmp(argv[1], "-d")==0)){
        //only the parameter code is changed
        strcat(parameter,argv[2]);
        strcat(request, "neuroscience");
    }

    //if there is ./api_req_par -d "pubmed" -q
    else if ((argc == 4) && (strcmp(argv[1], "-d")==0) && (strcmp(argv[3], "-q")==0)){
        //only the parameter code is changed
        strcat(parameter,argv[2]);
        strcat(request, "neuroscience");
    }

    //if there is ./api_req_par -d "pubmed" -q "neuroscience+intervention+learning"
    else if ((argc == 5) && (strcmp(argv[1], "-d")==0) && (strcmp(argv[3], "-q")==0)){
        //both the parameter and request codes are changed
        strcat(parameter,argv[2]);
        strcat(request, argv[4]);
    }

    //if there is ./api_req_par -q "neuroscience+intervention+learning"
    else if ((argc == 3) && (strcmp(argv[1], "-q")==0)){
        //only the request code is changed
        strcat(parameter,"pubmed");
        strcat(request, argv[2]);
    }

    //if there is ./api_req_par -q "neuroscience+intervention+learning" -d
    else if ((argc == 4) && (strcmp(argv[1], "-q")==0) && (strcmp(argv[3], "-d")==0)){
        //only the request code is changed
        strcat(parameter,"pubmed");
        strcat(request, argv[2]);
    }

    //if there is ./api_req_par -q "neuroscience+intervention+learning" -d "pubmed" 
    else if ((argc == 5) && (strcmp(argv[1], "-q")==0) && (strcmp(argv[3], "-d")==0)){
        //both the request and parameter codes are changed
        strcat(parameter,argv[4]);
        strcat(request, argv[2]);
    }

    else{
        printf("usage: ./api_req_par [-q] request [-d] parameter\n\n");
        printf("the api_req_par program is used to retrieve json data from the PubMed API\n\n");
        printf("optional arguments\n");
        printf("\t -q query        optional custom query; default is 'neuroscience'\n");
        printf("\t -d parameter    optional custom database code; default is 'pubmed',  see: https://www.ncbi.nlm.nih.gov/books/NBK25499/\n");
        return -1;
    }

    //initializes the CURL HTTP connection
    CURL *curl = curl_easy_init();

    //bits of the url that are joined together later
    char api[] = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=";                                                                     
    char type1[] = "&";
    char type2[] = "term=";
    char type3[] = "&retmode=json";                           
    char url[1000];

    //checks if CURL initialization is a success or not
    if (!curl){                                                                                         
        fprintf(stderr, "init failed\n");
        return EXIT_FAILURE;
    }
        
    //combines all the bits to produce a functioning url
    sprintf(url, "%s%s%s%s%s%s", api, parameter, type1 , type2, request, type3);                                             
                                          

    //sets the url to which the HTTP request will be sent to
    //first parameter is for the initialized curl HTTP request, second for the option to be set, and third for the value to be set
    curl_easy_setopt(curl, CURLOPT_URL, url);

    //if result is not retrieved then output error
    CURLcode result = curl_easy_perform(curl);

    //if result is not retrieved then output error
    if (result != CURLE_OK){                                                                            
        fprintf(stderr, "download problem: %s\n", curl_easy_strerror(result));
    }

    //deallocates memory for the CURL connection
    curl_easy_cleanup(curl);                                                                            
    return EXIT_SUCCESS;
}

The folowing program is run, and an executable is created after using the following command:

In [ ]:
!make

The default parameter is "pubmed" and the default requests are "neuroscience"

The folowing program is run, and an executable is created after using the following command:  

In [20]:
!./api_req_par| jq '.'

{
  "header": {
    "type": "esearch",
    "version": "0.3"
  },
  "esearchresult": {
    "count": "576170",
    "retmax": "20",
    "retstart": "0",
    "idlist": [
      "37489579",
      "37489574",
      "37489570",
      "37489566",
      "37489562",
      "37489558",
      "37489555",
      "37489510",
      "37489491",
      "37489403",
      "37489378",
      "37489373",
      "37489359",
      "37489358",
      "37489354",
      "37489326",
      "37489297",
      "37489273",
      "37489152",
      "37489105"
    ],
    "translationset": [
      {
        "from": "neuroscience",
        "to": "\"neuroscience's\"[All Fields] OR \"neurosciences\"[MeSH Terms] OR \"neurosciences\"[All Fields] OR \"neuroscience\"[All Fields]"
      }
    ],
    "querytranslation": "\"neuroscience s\"[All Fields] OR \"neurosciences\"[MeSH Terms] OR \"neurosciences\"[All Fields] OR \"neuroscience\"[All Fields]"
  }
}


In [21]:
!./api_req_par -q "aspirin" -d "pccompound" | jq '.'

{
  "header": {
    "type": "esearch",
    "version": "0.3"
  },
  "esearchresult": {
    "count": "126",
    "retmax": "20",
    "retstart": "0",
    "idlist": [
      "139196123",
      "139196122",
      "139196121",
      "138319320",
      "134716626",
      "132282528",
      "131953074",
      "131750206",
      "131716973",
      "129682947",
      "129672411",
      "123131972",
      "102337568",
      "91820534",
      "91820043",
      "91819941",
      "90478514",
      "86676097",
      "78401031",
      "77845952"
    ],
    "translationset": [],
    "translationstack": [
      {
        "term": "aspirin[All Fields]",
        "field": "All Fields",
        "count": "180",
        "explode": "N"
      },
      "GROUP"
    ],
    "querytranslation": "aspirin[All Fields]"
  }
}


The number of returned IDs can be adjusted with the retmax paramater:

In [22]:
!./api_req_par -q "neuroscience+intervention+learning&retmax=25" | jq '.esearchresult.idlist'

[
  "37488554",
  "37487822",
  "37487320",
  "37486497",
  "37485692",
  "37485663",
  "37484979",
  "37484917",
  "37483584",
  "37483354",
  "37483038",
  "37481676",
  "37481666",
  "37480442",
  "37480080",
  "37479991",
  "37479917",
  "37479341",
  "37479244",
  "37477974",
  "37476322",
  "37476302",
  "37476079",
  "37476005",
  "37475187"
]


In [23]:
!./api_req_par -q "neuroscience+intervention+learning&retmax=25" | jq '.esearchresult.idlist | length'

25


We can also use the query to search for an author.

we will add ```[au]``` after the name to specify it is an author

In [24]:
!./api_req_par -q "Darwin[au]" | jq '.esearchresult.count'

"630"


We get the idlist for the custom request:

In [25]:
!./api_req_par -q "Coral+Reefs&retmode=json&usehistory=y&sort=pub+date" | jq '.esearchresult.idlist'

[
  "37393678",
  "37315600",
  "37209734",
  "37290662",
  "37286001",
  "37257610",
  "37247740",
  "37286027",
  "37399735",
  "37385181",
  "37331272",
  "37311517",
  "37137368",
  "37105476",
  "37022443",
  "36549653",
  "37465983",
  "37487981",
  "37481620",
  "37100135"
]


Searching based on publication types:

we can do this by adding **AND** into the search
```
term=<searchQuery>+AND+filter[filterType]
```
```[pt]``` specifies that the filter type is publication type

More filters can be found at https://pubmed.ncbi.nlm.nih.gov/help/

In [26]:
!./api_req_par -q "stem+cells+AND+clinical+trial[pt]" | jq '{esearchresult: .esearchresult}'

{
  "esearchresult": {
    "count": "6355",
    "retmax": "20",
    "retstart": "0",
    "idlist": [
      "37445825",
      "37443807",
      "37426430",
      "37414012",
      "37408043",
      "37380983",
      "37377666",
      "37365605",
      "37355369",
      "37353884",
      "37345256",
      "37331950",
      "37316949",
      "37285522",
      "37267913",
      "37245011",
      "37244942",
      "37221247",
      "37218316",
      "37206071"
    ],
    "translationset": [
      {
        "from": "stem cells",
        "to": "\"stem cells\"[MeSH Terms] OR (\"stem\"[All Fields] AND \"cells\"[All Fields]) OR \"stem cells\"[All Fields]"
      }
    ],
    "querytranslation": "(\"stem cells\"[MeSH Terms] OR (\"stem\"[All Fields] AND \"cells\"[All Fields]) OR \"stem cells\"[All Fields]) AND \"clinical trial\"[Publication Type]"
  }
}


### 4. PubMed API metadata visualization

#### Frequency of topic sortpubdate field
Extracting the sortpubdate field for a “hydrogel drug” search results, limited to publication type clinical trials:

In [27]:
!./api_req_par -q "hydrogel+drug+AND+clinical+trial[pt]&sort=pub+date&retmax=500" | jq '.esearchresult.idlist[0:10]'

[
  "36418469",
  "36870516",
  "36842739",
  "36203046",
  "36261491",
  "35830550",
  "34653384",
  "35556170",
  "35413602",
  "35041809"
]


In [28]:
!./api_req_par -q "hydrogel+drug+AND+clinical+trial[pt]&sort=pub+date&retmax=500" | jq '.esearchresult.idlist | length'

302


The following code will store the list of IDs in a text file:

In [29]:
!./api_req_par -q "hydrogel+drug+AND+clinical+trial[pt]&sort=pub+date&retmax=500" | jq '.esearchresult.idlist' > idList.txt

To format the text file we use:

In [30]:
!cat idList.txt | tr -d '",[]' > idList2.txt

In [31]:
!sed -i '/^$/d' idList2.txt

In [32]:
!cat idList2.txt | wc -l

302


Show the first 10 IDs:

In [33]:
!head -10 idList2.txt

  36418469
  36870516
  36842739
  36203046
  36261491
  35830550
  34653384
  35556170
  35413602
  35041809


We want to get the E-summary of each of the IDs:

Hence we copy the api_call program from our previous project to our current directory

In [34]:
!cp ../basic_api_call/api_call .

We test to see if we get the date for one ID

In [35]:
!./api_call -i 34813072 | jq '.["result"]["34813072"]["sortpubdate"][0:10]'

"2022/01/01"


We then do the same to all the IDs and store them in a .txt file

In [36]:
%%bash
while read id; do
  ./api_call -i "$id" \
  | jq --arg ids "$id" '.["result"][$ids]["sortpubdate"][0:10]' \
  >> date_time.txt
  sleep 1
done < idList2.txt

In [37]:
!head -10 date_time.txt

"2023/07/01"
"2023/05/01"
"2023/05/01"
"2022/12/01"
"2022/10/19"
"2022/10/01"
"2022/10/01"
"2022/08/01"
"2022/06/01"
"2022/05/01"


#### Frequency of publication for an author search

In [38]:
!./api_req_par -q "Reed+LK[au]&sort=pub+date&retmax=500" | jq '.["esearchresult"]["count"]'

"59"


We store the id list data in a .txt file

In [39]:
!./api_req_par -q "Reed+LK[au]&sort=pub+date&retmax=500" | jq '.["esearchresult"]["idlist"]' > id_list3.txt

To format the text file we use:

In [40]:
!cat id_list3.txt | tr -d '",[]' > idList4.txt

In [41]:
!sed -i '/^$/d' idList4.txt

In [42]:
!cat idList4.txt | wc -l

59


Show the first 10 IDs:

In [43]:
!head -10 idList4.txt

  37302379
  36871651
  37292993
  36468157
  35691520
  36061313
  35856017
  34801137
  34786536
  34425636


In [44]:
%%bash
while read id; do
  ./api_call -i "$id" \
  | jq --arg ids "$id" '.["result"][$ids]["sortpubdate"][0:10]' \
  >> date_time2.txt
  sleep 1
done < idList4.txt